In [2]:
import sys
import pandas as pd
import numpy as np
import sklearn

In [58]:
# 读取数据，

base_path = 'dataset_temp/okxorderbook20240421_1.csv'
data_all = pd.read_csv(base_path)
data_all.head()

# 把time_level裁剪掉 level
result = data_all['time_level'][0].replace(str(data_all['level'][0]), '')
# 减去level，得到标准时间戳
data_all['time'] = data_all.apply(lambda x: x['time_level'].replace(str(x['level']), ''), axis=1)
# 把data_all['time']转换为时间戳 long 类型
data_all['time'] =data_all['time'].astype(np.int64)

# 标记一下是买单还是卖单，卖单为1 ，卖单为-1
data_all['side'] = data_all['level'].apply(lambda x: 1 if x > 0 else 0)
# 调整列的顺序

# 排序 先按time从小到大，如果time相同，按level从大到小
data_all = data_all.sort_values(by=['server_time', 'level'], ascending=[True, False])

# 调整顺序
data_all = data_all[['server_time', 'level', 'price', 'amount','side']]
data_all.to_csv('dataset_temp/test1.csv', index=False)
data_all.head()

,server_time,level,price,amount,side
32863,2024-04-21 00:00:00.004,50,64824.3,0.113338,1
32861,2024-04-21 00:00:00.004,49,64824.2,0.000771,1
32860,2024-04-21 00:00:00.004,48,64824.0,0.144443,1
32859,2024-04-21 00:00:00.004,47,64823.9,0.097747,1
32858,2024-04-21 00:00:00.004,46,64823.8,0.089804,1


In [59]:
# 裁剪数据，只去买卖个三个档
data_all = data_all[data_all['level'] <= 3]
data_all = data_all[data_all['level'] >= -3]
data_all.head(10)
data_all.to_csv('dataset_temp/test2.csv', index=False)


In [88]:
data_all

,server_time,level,price,amount,side
32840,2024-04-21 00:00:00.004,3,64810.0,0.076300,1
32829,2024-04-21 00:00:00.004,2,64808.1,0.319200,1
32818,2024-04-21 00:00:00.004,1,64806.1,0.595363,1
32768,2024-04-21 00:00:00.004,-1,64806.0,0.568911,0
32779,2024-04-21 00:00:00.004,-2,64805.8,0.003954,0
...,...,...,...,...,...
99863,2024-04-21 12:01:42.306,-2,65176.4,0.220000,0
99874,2024-04-21 12:01:42.306,-3,65176.1,0.005050,0
99952,2024-04-21 12:01:42.506,-1,65176.7,0.302038,0
99963,2024-04-21 12:01:42.506,-2,65176.4,0.220000,0


In [107]:
# 把同一时间的数据打平
asks = data_all[data_all['side'] == 1]
asks_pivot = asks.pivot(index='server_time', columns='level', values='price')
asks_columns = asks_pivot.columns.to_list()
asks_pivot.columns =[f'ask_{i}_price' for i in asks_pivot.columns]

bids = data_all[data_all['side'] == 0]
bids_pivot = bids.pivot(index='server_time', columns='level', values='price')
bids_pivot.columns =[f'bid_{abs(i)}_price' for i in bids_pivot.columns]

# 合并bid和ask订单
data_all = pd.concat([bids_pivot, asks_pivot], axis=1).reset_index()

# 打印结果
data_all.head()


,server_time,bid_3_price,bid_2_price,bid_1_price,ask_1_price,ask_2_price,ask_3_price
0,2024-04-21 00:00:00.004,64804.8,64805.8,64806.0,64806.1,64808.1,64810.0
1,2024-04-21 00:00:00.204,64805.5,64805.8,64806.0,64806.1,64808.0,64808.1
2,2024-04-21 00:00:00.404,64805.5,64805.8,64806.0,64806.1,64808.0,64808.1
3,2024-04-21 00:00:00.604,64805.5,64805.8,64806.0,64806.1,64807.9,64808.1
4,2024-04-21 00:00:00.804,64805.5,64805.8,64806.0,64806.1,64807.9,64808.1
